# DSCI 572 Lab 2


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import time

import scipy.stats
import scipy.optimize as spo
import scipy.special
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor
from sklearn.preprocessing import scale, StandardScaler, minmax_scale, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

## Instructions
rubric={mechanics:20}

Follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).

## Exercise 1: floating point errors

For each of the code snippets below, explain the result. The first three are answered for you, so you can get a sense of what types of explanations we're looking for. Note that in both the second and third case we get the "expected" result (zero), but the explanations are very different.

In [2]:
0.3 - 0.2 - 0.1

-2.7755575615628914e-17

**Sample solution**: The result is not zero because 0.3, 0.2, and 0.1 are not represented exactly as floating point numbers.

In [3]:
0.5 - 0.25 - 0.125 - 0.125

0.0

**Sample solution**: The result is zero because 0.5, 0.25, and 0.125 are powers of 2 and there they _are_ represented exactly as floating point numbers. There is no roundoff error present. 

In [4]:
0.4 - 0.2 - 0.2

0.0

**Sample solution**: The result is correct (zero) despite the fact that 0.4 and 0.2 are not represented exactly as floating point numbers. This is a case of good luck: while 0.4 and 0.2 are not represented exactly, the roundoff errors happened to cancel out during the subtractions.

### snippet (a)
rubric={reasoning:5}

In [ ]:
30 - 20 - 10

### snippet (b)
rubric={reasoning:5}

In [ ]:
30.0 - 20.0 - 10.0

### snippet (c)
rubric={reasoning:5}

In [ ]:
(10.0**100 + 1) == 10.0**100

### snippet (d)
rubric={reasoning:5}

In [ ]:
(10.0**100000 + 1) - 10.0**100000

### snippet (e)
rubric={reasoning:5}

In [ ]:
np.exp(1000) - np.exp(1000)

### snippet (f)
rubric={reasoning:5}

In [ ]:
1/np.exp(1000) == np.exp(-1000)

### snippet (g)
rubric={reasoning:5}

In [ ]:
1/np.exp(100) == np.exp(-100)

### snippet (h)
rubric={reasoning:5}

In [ ]:
np.exp(1000)==np.exp(10000)

### snippet (i)
rubric={reasoning:5}

In [ ]:
sum(np.zeros(10)+0.1)

### snippet (j)
rubric={reasoning:5}

In [ ]:
np.sin(np.pi)

### (optional) snippet (k)
rubric={reasoning:1}

In [ ]:
x = np.ones(100000)
x[0] = 1e20

y = np.ones(100000)
y[-1] = 1e20

sum(x) == sum(y)

### (optional) snippet (l)
rubric={reasoning:1}

In [ ]:
x = np.zeros(10)+0.1
sum(x) == np.sum(x)

Hint for the above: see [Pairwise summation](https://en.wikipedia.org/wiki/Pairwise_summation). 

## (optional) Exercise 2: floating point max/min
rubric={reasoning:2}

As discussed in lecture, IEEE double precision floating point numbers use 53 bits for the mantissa (one of which is a sign bit) and 11 bits for the exponent (again, one of which is a sign bit). Given thus, calculate the largest (furthest from zero) and smallest (closest to zero) possible representable floating point numbers. Then empirically check your results with Python. Are they what you expected? Discuss.

NOTE: Python has a special behaviour that we need to watch out for. If you do something like `10**1000` you will get a giant integer. That's because Python has a dynamically expanding integer type. This has nothing to do with floating point representations, which are the thing we really care about for scientific computation (not to mention that most other languages, including R, do not do this). So, when playing around, make sure you write `10**1000.0` or `10.0*1000` to ensure it's a floating point. Or `1e1000`... but that doesn't work for other bases besides 10. 

(Also, and this is _REALLY_ out of scope but just FYI if anyone cares, in some languages `1eX` and `10^X` will return slightly different answers, if the language uses a special routine for `1eX` that is more optimized than the generral power function. I cannot imagine this ever mattering to any of us.)

## Exercise 3: logpdf
rubric={reasoning:10}

In most scientific computing environments, such as R or NumPy/SciPy, there are functions to compute the probability density function (pdf) of a probability distribution, and separate functions to compute the log of the pdf. For example, looking at the [R normal distribution documentation](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Normal.html), you will see that you can set `log=TRUE` or `log=FALSE`. Likewise in Python we have `scipy.stats.norm.pdf` and `scipy.stats.norm.logpdf`. And yet, why bother creating these extra functions, when we can just take the log ourselves?! For example, we have NumPy functions for $\sin(x)$ and $\cos(x)$ but not for $\log\sin(x)$ and $\log \cos (x)$.  

In [5]:
scipy.stats.norm.logpdf(1) # is function this useless?

-1.4189385332046727

In [6]:
np.log(scipy.stats.norm.pdf(1)) # seems to do the same thing!

-1.4189385332046727

### (optional) 3(b)
rubric={accuracy:1}

Write a numerically safe function `log_gaussian_pdf` that computes the log of the standard (zero mean, unit variance) Gaussian PDF. Your function should produce the same result as `scipy.stats.norm.logpdf` when evaluated at $x=100$.

## (optional) Exercise 4: $\log(1+\exp(z))$, continued
rubric={accuracy:1,reasoning:1}

In lecture we discussed computing $\log(1+\exp(z))$. We wrote a better version of the function for when $z\gg1$, reproduced below.

In [8]:
def log_1_plus_exp(z):
    return np.log(1+np.exp(z))

def log_1_plus_exp_safe(z):
    if z > 100:
        return z
    else:
        return np.log(1+np.exp(z))

Now let's consider the case of $z\ll -1$, i.e. when $z$ is a large negative number. In that case, we get zero with both of the above implementations:

In [9]:
print(log_1_plus_exp(-100))

0.0


In [10]:
print(log_1_plus_exp_safe(-100))

0.0


Your tasks:

1. Investigate why this is happening. Is the problem that $\exp(-100)$ itself underflows?
2. Write a function `log_1_plus_exp_safer` that works well when $z\gg 1$ and $z\ll -1$.
3. For what range of values of $z$ does your `log_1_plus_exp_safer` function give reasonable results?
4. Your code presumably contains two thresholds, the upper and lower cutoffs for $z$ at which the approximations are invoked. Can you reason about the "optimal" or "best" values for these thresholds?

## Exercise 5: softmax logistic regression and log-sum-exp
rubric={reasoning:15,accuracy:5}

In the "multinomial" (aka softmax) approach to multi-class logistic regression, your loss ends up having one term for each class, so you get something of the form $\log \sum_{k=1}^c \exp(z_k)$, where $c$ is the number of classes, and $z_k$ is a quantity involving the weights and training data. For any choice of a constant $a$, we can rewrite this as follows:

$$\begin{align}\log \displaystyle\sum_{k=1}^c \exp(z_k) &= \log \displaystyle\sum_{k=1}^c \exp(z_k-a+a)\\ &= \log \left( \displaystyle\sum_{k=1}^c \exp(z_k-a)\exp(a) \right) \\ &= \log \left( \exp(a)\displaystyle\sum_{k=1}^c \exp(z_k-a)\right) \\ &= \log \left( \exp(a) \right) + \log\displaystyle\sum_{k=1}^c \exp(z_k-a) \\ &= a+ \log \displaystyle\sum_{k=1}^c \exp(z_k-a)\end{align}$$

Note: you only need to look at the first and last expression - the middle parts are only there to convince you they are indeed equivalent.

1. Explain why this final expression might be more numerically stable and why $a=\max \{z_1,z_2,\ldots,z_c\}$ is a sensible choice.
2. If $a=\max \{z_1,z_2,\ldots,z_c\}$, this trick seems to rely on the fact that overflow is more of a danger than underflow, because we may now compute $exp$ of some very large negative values if $a$ is large. Explain why overflow is more of a problem than underflow in this calculation.
3. Write a python function `log_sum_exp` that takes in an array `z` and computes the _original_ expression. Write another function `log_sum_exp_stable` that computes the final (more stable) expression using $a=\max \{z_1,z_2,\ldots,z_c\}$. Discuss the behaviour of the two functions. Also, compare your implementation to In fact, [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html#scipy.special.logsumexp) for one or two cases.
4. (optional) Earlier, we used the approximation $\log(1+\exp(z))\approx z$ when $z\gg 1$. Is that just a specific case of what we have here? It seems that what we did earlier was an approximation, whereas what we did here is mathematically exact. Do you think there is any significance to this distinction, in practice?

(FYI: you'll see this trick in real implementations of ML algorithms! )

## Exercise 6: SGD implementation

Below is a Python function that performs gradient descent, much like the code you wrote in lab 1. I just made a few small changes:

- Removed the docstring and `print` statements for brevity.
- Renamed `x` to `w`.
- In this version, the user specifies a fixed number of iterations `n_iters` instead of a tolerance $\epsilon$.
- In this version, the user passes in `X` and `y` explicitly, and these are passed to `f_grad`.

In [11]:
def gradient_descent(f, f_grad, w0, X, y, n_iters=1000, α=0.001):
    w = w0
    for t in range(n_iters):
        w = w - α*f_grad(w, X, y)
    return w

The code works (although is very slow as per usual for gradient descent). For example, here we do ordinary least squares linear regression:

In [12]:
from sklearn.datasets import load_boston

In [13]:
data = load_boston()
X = data['data']
y = data['target']
X = minmax_scale(X)

In [14]:
def loss_ols(w,X,y): return 0.5*np.sum((X@w-y)**2)
def loss_ols_grad(w,X,y): return X.T@(X@w) - X.T@y

w0 = np.zeros(X.shape[1])

In [15]:
w_gd = gradient_descent(loss_ols, loss_ols_grad, w0, X, y, n_iters=10**5)

In [16]:
w_gd

array([ -7.37446749,   4.79336123,   3.71903592,   2.67885881,
        -2.90900155,  36.50466055,   2.10874674,  -4.55318591,
         5.45719841,  -6.02418944,  -4.39530169,   9.88901723,
       -11.79090734])

In [17]:
lr = LinearRegression(fit_intercept=False).fit(X, y)

In [18]:
lr.coef_

array([ -7.37446749,   4.79336123,   3.71903592,   2.67885881,
        -2.90900155,  36.50466055,   2.10874674,  -4.55318591,
         5.45719841,  -6.02418944,  -4.39530169,   9.88901723,
       -11.79090734])

As we can see, the coefficients obtained from gradient descent are very similar to those obtained by scikit-learn. All is well so far.

#### 6(a)
rubric={accuracy:20,quality:10}

Your task is to implement a function `stochastic_gradient_descent`, that performs SGD, _using_ the `gradient_descent` function provided above. You can have your function accept the same arguments as the `gradient_descent` function above, except:

- Change `n_iters` to `n_epochs`.
- Add an extra `batch_size` argument.

Note: even though this might not be a good idea in general, you can leave $\alpha$ constant in this implementation; that is, you can use the same $\alpha$ across all iterations. 

Note: the pedagogical goal here is to help you see how SGD relates to regular gradient descent. In reality it would be fine to implement SGD "from scratch" without calling a GD function.

#### 6(b)
rubric={reasoning:5}

Show that when the batch size is set to the whole training set, you get exactly the same results as with gradient descent.

## Exercise 7: `SGDClassifier` and `SGDRegresor`

In this exercise we'll explore training a classifier with SGD on the [Sentiment140 dataset](http://help.sentiment140.com/home), which contains tweets labeled with sentiment associated with a brand, product, or topic. Please start by doing the following:

1. Download the corpus from [here](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip).
2. Unzip.
3. Copy the file `training.1600000.processed.noemoticon.csv` into the current directory.
4. Create a `.gitignore` file so that you don't accidentally commit the dataset.

Once you're done the above, steps, run the starter code below:

In [ ]:
# Data loading and preprocessing
tweets_df = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                        encoding = "ISO-8859-1",
                        names=["label","id", "date", "no_query", "name", "text"])
tweets_df['label'] = tweets_df['label'].map({0: 'neg', 4: 'pos'})

tweets_df.head()

# only consider positive and negative reviews
tweets_pos_neg_df = tweets_df[tweets_df['label'].str.startswith(('pos','neg'))]

In [ ]:
tweets_pos_neg_df.head()

Now we split the data:

In [ ]:
tweets_df_train, tweets_df_test = train_test_split(tweets_pos_neg_df)

And then we encode it using `CountVectorizer`, which may take a minute or so:

In [ ]:
vec = CountVectorizer(stop_words='english')

X_train = vec.fit_transform(tweets_df_train['text']) 
y_train = tweets_df_train['label']

# just use the already-fit vectorizer
X_test = vec.transform(tweets_df_test['text']) 
y_test = tweets_df_test['label']

Note that our training data is rather large compared to datasets we've explored in the past:

In [ ]:
X_train.shape

In [ ]:
type(X_train)

Here is the fraction of elements that are nonzero. Having a sparse matrix really helps!!

In [ ]:
X_train.nnz/np.prod(X_train.shape)

Now let's train a classifier. I'll use `time` instead of `%timeit` because I want to keep the output, and it gets lost with `%timeit`.

In [ ]:
lr = LogisticRegression()

In [ ]:
t = time.time()

lr.fit(X_train, y_train)

print("Training took %.1f seconds" % (time.time() - t));

In [ ]:
lr.score(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

#### 7(a)
rubric={reasoning:10}

Train a logistic regression model on the same dataset using [`SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) instead of `LogisticRegression`. Compare the training time.

FYI: there's also a regression version, [`SGDRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html), which does linear regression with SGD. However, we won't explore `SGDRegressor` here.

#### 7(b)
rubric={reasoning:15}

Discuss the training and test accuracies of the two approaches. Can you explain what you're seeing?

#### (optional) 7(c)
rubric={reasoning:2}

One possible explanation for `SGDClassifier`'s speed is that it's just doing fewer iterations/epochs. `SGDClassifier` and `LogisticRegression` have an `n_iter_` attribute which you can check after fitting. Compare these values and discuss them in the context of the above hypothesis. Then, using the `max_iter` parameter, do a "fair" experiment where `SGDClassifier` and `LogisticRegression` do the same number of passes through the dataset, and comment on the results.